In [1]:
# General Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from selenium import webdriver
import main
import seaborn as sns
import chromedriver_autoinstaller
from datetime import datetime
import matplotlib.pyplot as plt
from mplsoccer import Pitch, VerticalPitch
from matplotlib.colors import to_rgba
import matplotlib as mpl
from collections import defaultdict
import pickle

In [2]:
chromedriver_autoinstaller.install()

import warnings
warnings.filterwarnings('ignore')

## Pull Data

In [3]:
chromedriver_autoinstaller.install()

if __name__ == "__main__":
    driver = webdriver.Chrome()
    
# whoscored match centre url of the required match (Example: Barcelona vs Sevilla)
url = "https://www.whoscored.com/Matches/1640972/Live/England-Premier-League-2022-2023-Tottenham-Chelsea"
match_data = main.getMatchData(driver, url, close_window=True)

# Match dataframe containing info about the match
matches_df = main.createMatchesDF(match_data)

# Events dataframe      
events_df = main.createEventsDF(match_data)

# match Id
matchId = match_data['matchId']

# Information about respective teams as dictionary
home_data = matches_df['home'][matchId]
away_data = matches_df['away'][matchId]
score = matches_df['score'][matchId]
score = ' ' + score.replace(':', '-') + ' '

home_name = home_data['name']
away_name = away_data['name']
date = match_data['startDate'].split('T')[0]
date_ = datetime.strptime(date, '%Y-%m-%d').date()
date_str = date_.strftime('%d/%m/%Y')

Region: England, League: Premier League, Season: 2022/2023, Match Id: 1640972


In [4]:
homeID = home_data['teamId']
awayID = away_data['teamId']

In [5]:
home_color = '#9C9C9C'
away_color = '#2D5DE7'

## xThreat
- Want team values for 3 xThreat - xThreat via Pass & Carry, and xThreat received (i.e. ability to produce and receive)
- Using Karun Singh's values

In [6]:
zones = []
for i in range(8):
    row = []
    for j in range(12):
        zone = i + ((j)*8)
        row.append(zone)
    zones.append(row)
zones = np.array(zones)
zones

array([[ 0,  8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88],
       [ 1,  9, 17, 25, 33, 41, 49, 57, 65, 73, 81, 89],
       [ 2, 10, 18, 26, 34, 42, 50, 58, 66, 74, 82, 90],
       [ 3, 11, 19, 27, 35, 43, 51, 59, 67, 75, 83, 91],
       [ 4, 12, 20, 28, 36, 44, 52, 60, 68, 76, 84, 92],
       [ 5, 13, 21, 29, 37, 45, 53, 61, 69, 77, 85, 93],
       [ 6, 14, 22, 30, 38, 46, 54, 62, 70, 78, 86, 94],
       [ 7, 15, 23, 31, 39, 47, 55, 63, 71, 79, 87, 95]])

In [7]:
xT_list = []
path = "https://karun.in/blog/data/open_xt_12x8_v1.json"
xT = pd.read_json(path)
xT = np.array(xT).T
xT_rows, xT_cols = xT.shape

x_bins = np.array(np.linspace(0,100, 13), dtype=object)
y_bins = np.array(np.linspace(0, 100, 9), dtype=object)

In [8]:
df_passes = events_df[(events_df['type'] == 'Pass') & 
                      (events_df['outcomeType'] == 'Successful') & 
                      (events_df['teamId'] == homeID) & 
                      (events_df['passFreekick'] == False) & 
                      (events_df['passCorner'] == False) &
                      (events_df['throwIn'] == False)].reset_index(drop=True)

df_passes['x1_bin'] = pd.cut(df_passes['x'], bins=x_bins, labels=False, include_lowest=True)
df_passes['x2_bin'] = pd.cut(df_passes['endX'], bins=x_bins, labels=False, include_lowest=True)
df_passes['y1_bin'] = pd.cut(df_passes['y'], bins=y_bins, labels=False, include_lowest=True)
df_passes['y2_bin'] = pd.cut(df_passes['endY'], bins=y_bins, labels=False, include_lowest=True)
df_passes['xT_start'] = df_passes[['x1_bin', 'y1_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
df_passes['xT_end'] = df_passes[['x2_bin', 'y2_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
df_passes['xT_diff'] = df_passes['xT_end'] - df_passes['xT_start']
df_passes['zone'] = df_passes[['x1_bin', 'y2_bin']].apply(lambda x:zones[x[1]][x[0]], axis=1)
df_passes = df_passes[df_passes['xT_diff'] > 0].reset_index(drop=True)
df_passes = df_passes[['x', 'y', 'x1_bin', 'y1_bin', 'zone', 'xT_diff']]
df_passes

,x,y,x1_bin,y1_bin,zone,xT_diff
0,22.4,95.9,2,7,22,0.000337
1,8.5,85.1,1,6,12,0.000625
2,3.7,61.1,0,4,0,0.000366
3,44.9,67.5,5,5,40,0.000445
4,69.5,7.3,8,0,64,0.013629
...,...,...,...,...,...,...
169,29.2,92.6,3,7,29,0.001328
170,21.9,58.5,2,4,19,0.000662
171,8.1,47.0,0,3,2,0.000367
172,33.0,87.1,3,6,31,0.004141


In [9]:
df_pass_xt = pd.DataFrame(df_passes.groupby('zone')['xT_diff'].sum()).reset_index()
df_pass_xt

,zone,xT_diff
0,0,0.000366
1,1,0.003534
2,2,0.000367
3,3,0.000755
4,4,0.001417
...,...,...
62,71,0.003712
63,73,0.001970
64,74,0.054691
65,76,0.006561


## Carries 

In [10]:
df_carries = pd.DataFrame()
min_len = 3
max_len = 60
min_time = 1
max_time = 12

for idx, match_event in events_df.iterrows():
    if idx == len(events_df) - 1:
        break
    
    cur_team = match_event['teamId']
    idx_next = idx + 1
    events_df_next = events_df.loc[idx_next]
    take_ons = 0
    loop = True
    count = 0 
    
    while loop:
        next_events = events_df.loc[idx_next]
        
        if next_events['type'] == 'TakeOn' and next_events['outcomeType'] == 'Successful':
            take_ons += 1
            loop = True
        
        elif (next_events['type'] == 'TakeOn' and next_events['outcomeType'] == 'Unsuccessful') or (next_events['teamId'] != cur_team and next_events['type'] == 'Challenge' and next_events['outcomeType'] == 'Unsuccessful') or (next_events['type'] == 'Foul'):
            loop = True
            
        else:
            loop = False
            
        idx_next += 1
        count += 0 
        

    same_team = cur_team == next_events['teamId']
    not_ball_touch = match_event['type'] != 'BallTouch'
    dx = 105 * (match_event['endX'] - next_events['x'])/100
    dy = 68 * (match_event['endY'] - next_events['y'])/100
    min_dist = dx**2 + dy**2 >= min_len**2
    max_dist = dx**2 + dy**2 <= max_len**2
    same_period = match_event['period'] == next_events['period']
    
    valid_carry = same_team & not_ball_touch & min_dist & max_dist & same_period 
    
    if valid_carry:
        carry = pd.DataFrame()
        prev = match_event
        nxt = next_events
        carry.loc[0, 'eventId'] = prev['eventId'] + count
        carry['id'] = prev['id'] + count
        carry['teamId'] = nxt['teamId']
        carry['x'] = prev['endX']
        carry['y'] = prev['endY']
        carry['period'] = nxt['period']
        carry['type'] = carry.apply(lambda x: {'value': 99, 'displayName': 'Carry'}, axis=1)
        carry['outcomeType'] = 'Successful'
        carry['qualifiers'] = carry.apply(
            lambda x: {'type': {'value': 999, 'displayName': 'takeOns'}, 'value': str(take_ons)}, axis=1)
        carry['satisfiedEventsTypes'] = carry.apply(lambda x: [], axis=1)
        carry['isTouch'] = True
        carry['playerId'] = nxt['playerId']
        carry['playerName'] = nxt['playerName']
        carry['endX'] = nxt['x']
        carry['endY'] = nxt['y']
        
        df_carries = pd.concat([df_carries, carry], ignore_index=True, sort=False)


In [11]:
df_carries1 = df_carries[df_carries['teamId'] == homeID]
df_carries1['x1_bin'] = pd.cut(df_carries1['x'], bins=x_bins, labels=False, include_lowest=True)
df_carries1['x2_bin'] = pd.cut(df_carries1['endX'], bins=x_bins, labels=False, include_lowest=True)
df_carries1['y1_bin'] = pd.cut(df_carries1['y'], bins=y_bins, labels=False, include_lowest=True)
df_carries1['y2_bin'] = pd.cut(df_carries1['endY'], bins=y_bins, labels=False, include_lowest=True)
df_carries1['xT_start'] = df_carries1[['x1_bin', 'y1_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
df_carries1['xT_end'] = df_carries1[['x2_bin', 'y2_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
df_carries1['xT_diff'] = df_carries1['xT_end'] - df_carries1['xT_start']
df_carries1['zone'] = df_carries1[['x1_bin', 'y2_bin']].apply(lambda x:zones[x[1]][x[0]], axis=1)
df_carries1 = df_carries1[df_carries1['xT_diff'] > 0].reset_index(drop=True)
df_carries1 = df_carries1[['x', 'y', 'x1_bin', 'y1_bin', 'zone', 'xT_diff']]
df_carries1

,x,y,x1_bin,y1_bin,zone,xT_diff
0,56.9,4.3,6,0,48,0.006485
1,28.5,83.2,3,6,31,0.000668
2,58.2,13.1,6,1,50,0.003233
3,25.0,84.2,2,6,22,0.001171
4,48.1,33.0,5,2,42,0.002565
...,...,...,...,...,...,...
74,75.4,7.0,9,0,72,0.010363
75,56.1,79.1,6,6,53,0.003233
76,31.2,93.3,3,7,31,0.001486
77,55.4,97.2,6,7,55,0.006485


In [12]:
df_carry_xt = pd.DataFrame(df_carries1.groupby('zone')['xT_diff'].sum()).reset_index()
df_carry_xt

,zone,xT_diff
0,1,0.001118
1,5,0.001379
2,8,0.000652
3,11,0.001050
4,12,0.001050
5,13,0.000236
6,14,0.000638
7,18,0.001092
8,19,0.000152
9,20,0.002469


In [13]:
df_xt = pd.DataFrame(pd.concat([df_pass_xt, df_carry_xt]).groupby('zone')['xT_diff'].sum()).reset_index()

# To scale
xT_max = df_xt['xT_diff'].max()
df_xt['xT_alpha'] = df_xt['xT_diff']/xT_max
df_xt

,zone,xT_diff,xT_alpha
0,0,0.000366,0.004302
1,1,0.004651,0.054672
2,2,0.000367,0.004312
3,3,0.000755,0.008873
4,4,0.001417,0.016651
...,...,...,...
72,75,0.085080,1.000000
73,76,0.006561,0.077119
74,77,0.026368,0.309917
75,78,0.000981,0.011527


## For Away Team 

In [14]:
df_passes2 = events_df[(events_df['type'] == 'Pass') & 
                      (events_df['outcomeType'] == 'Successful') & 
                      (events_df['teamId'] == awayID) & 
                      (events_df['passFreekick'] == False) & 
                      (events_df['passCorner'] == False) &
                      (events_df['throwIn'] == False)].reset_index(drop=True)

df_passes2['x1_bin'] = pd.cut(df_passes2['x'], bins=x_bins, labels=False, include_lowest=True)
df_passes2['x2_bin'] = pd.cut(df_passes2['endX'], bins=x_bins, labels=False, include_lowest=True)
df_passes2['y1_bin'] = pd.cut(df_passes2['y'], bins=y_bins, labels=False, include_lowest=True)
df_passes2['y2_bin'] = pd.cut(df_passes2['endY'], bins=y_bins, labels=False, include_lowest=True)
df_passes2['xT_start'] = df_passes2[['x1_bin', 'y1_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
df_passes2['xT_end'] = df_passes2[['x2_bin', 'y2_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
df_passes2['xT_diff'] = df_passes2['xT_end'] - df_passes2['xT_start']
df_passes2['zone'] = df_passes2[['x1_bin', 'y2_bin']].apply(lambda x:zones[x[1]][x[0]], axis=1)
df_passes2 = df_passes2[df_passes2['xT_diff'] > 0].reset_index(drop=True)
df_passes2 = df_passes2[['x', 'y', 'x1_bin', 'y1_bin', 'zone', 'xT_diff']]
df_pass_xt2 = pd.DataFrame(df_passes2.groupby('zone')['xT_diff'].sum()).reset_index()

df_carries2 = df_carries[df_carries['teamId'] == awayID]
df_carries2['x1_bin'] = pd.cut(df_carries2['x'], bins=x_bins, labels=False, include_lowest=True)
df_carries2['x2_bin'] = pd.cut(df_carries2['endX'], bins=x_bins, labels=False, include_lowest=True)
df_carries2['y1_bin'] = pd.cut(df_carries2['y'], bins=y_bins, labels=False, include_lowest=True)
df_carries2['y2_bin'] = pd.cut(df_carries2['endY'], bins=y_bins, labels=False, include_lowest=True)
df_carries2['xT_start'] = df_carries2[['x1_bin', 'y1_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
df_carries2['xT_end'] = df_carries2[['x2_bin', 'y2_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
df_carries2['xT_diff'] = df_carries2['xT_end'] - df_carries2['xT_start']
df_carries2['zone'] = df_carries2[['x1_bin', 'y2_bin']].apply(lambda x:zones[x[1]][x[0]], axis=1)
df_carries2 = df_carries2[df_carries2['xT_diff'] > 0].reset_index(drop=True)
df_carries2 = df_carries2[['x', 'y', 'x1_bin', 'y1_bin', 'zone', 'xT_diff']]
df_carry_xt2 = pd.DataFrame(df_carries2.groupby('zone')['xT_diff'].sum()).reset_index()

df_xt2 = pd.DataFrame(pd.concat([df_pass_xt2, df_carry_xt2]).groupby('zone')['xT_diff'].sum()).reset_index()
xT_max2 = df_xt2['xT_diff'].max()
df_xt2['xT_alpha'] = df_xt2['xT_diff']/xT_max2
df_xt2

,zone,xT_diff,xT_alpha
0,0,0.005822,0.023548
1,1,0.006708,0.027132
2,3,0.001417,0.005730
3,4,0.001417,0.005730
4,5,0.003194,0.012920
...,...,...,...
71,78,0.043158,0.174571
72,79,0.014575,0.058956
73,81,0.005819,0.023538
74,83,0.053140,0.214946


## Top 3 Metrics 

### xThreat via Pass  

In [15]:
home_players = events_df[events_df['teamId'] == homeID]['playerName'].unique().tolist()
home_players = [x for x in home_players if str(x) != 'nan']

df_passes_home = events_df[(events_df['type'] == 'Pass') & (events_df['outcomeType'] == 'Successful') & 
                           (events_df['teamId'] == homeID) & 
                           (events_df['playerName'].isin(home_players))].reset_index(drop=True)


xT_list = []
for name in home_players:
    test = df_passes_home[(df_passes_home['playerName'] == name) &
                          (df_passes_home['passFreekick'] == False) & 
                          (df_passes_home['passCorner'] == False) &
                          (df_passes_home['throwIn'] == False)]
    
    if len(test) > 0:
        test['x1_bin'] = pd.cut(test['x'], bins=x_bins, labels=False, include_lowest=True)
        test['x2_bin'] = pd.cut(test['endX'], bins=x_bins, labels=False, include_lowest=True)
        test['y1_bin'] = pd.cut(test['y'], bins=y_bins, labels=False, include_lowest=True)
        test['y2_bin'] = pd.cut(test['endY'], bins=y_bins, labels=False, include_lowest=True)
        test['xT_start'] = test[['x1_bin', 'y1_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
        test['xT_end'] = test[['x2_bin', 'y2_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
        test['xT_diff'] = test['xT_end'] - test['xT_start']
        test = test[test['xT_diff']>0].reset_index(drop=True)
        xT_added = test['xT_diff'].sum()

        xT_list.append([name, xT_added])
        
    else:
        xT_list.append([name, 0])

xT_pass_home = pd.DataFrame(xT_list)
xT_pass_home.rename({0:'playerName', 1:'xT'}, axis='columns', inplace=True)
xT_pass_home = xT_pass_home.sort_values(by='xT', ascending=False).reset_index(drop=True)[:3]
xT_pass_home

,playerName,xT
0,Pierre-Emile Højbjerg,0.103549
1,Emerson Royal,0.082355
2,Harry Kane,0.072758


### xThreat via Carry

In [16]:
xT_carry_list = []
for name in home_players:
    test = df_carries[df_carries['playerName'] == name]
    
    if len(test) > 0:
        test['x1_bin'] = pd.cut(test['x'], bins=x_bins, labels=False, include_lowest=True)
        test['x2_bin'] = pd.cut(test['endX'], bins=x_bins, labels=False, include_lowest=True)
        test['y1_bin'] = pd.cut(test['y'], bins=y_bins, labels=False, include_lowest=True)
        test['y2_bin'] = pd.cut(test['endY'], bins=y_bins, labels=False, include_lowest=True)
        test['xT_start'] = test[['x1_bin', 'y1_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
        test['xT_end'] = test[['x2_bin', 'y2_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
        test['xT_diff'] = test['xT_end'] - test['xT_start']
        test = test[test['xT_diff']>0].reset_index(drop=True)
        xT_added = test['xT_diff'].sum()

        xT_carry_list.append([name, xT_added])
        
    else:
        xT_carry_list.append([name, 0])
    
xT_carry_home = pd.DataFrame(xT_carry_list)
xT_carry_home.rename({0:'playerName', 1:'xT'}, axis='columns', inplace=True)
xT_carry_home = xT_carry_home.sort_values(by='xT', ascending=False).reset_index(drop=True)[:3]
xT_carry_home

,playerName,xT
0,Dejan Kulusevski,0.121777
1,Emerson Royal,0.080838
2,Ben Davies,0.051313


### xThreat Received  

In [17]:
# Add Recipient column
events_df["passRecipient"] = events_df['playerName'].shift(-1)
column_to_move = events_df.pop("passRecipient")
events_df.insert(29, "passRecipient", column_to_move)

df_passes_home = events_df[(events_df['type'] == 'Pass') & (events_df['outcomeType'] == 'Successful') & 
                           (events_df['teamId'] == homeID) & 
                           (events_df['playerName'].isin(home_players))].reset_index(drop=True)

xT_received_list = []
for name in home_players:
    test = df_passes_home[(df_passes_home['passRecipient'] == name) &
                          (df_passes_home['passFreekick'] == False) & 
                          (df_passes_home['passCorner'] == False) &
                          (df_passes_home['throwIn'] == False)]
    
    if len(test) > 0:
        test['x1_bin'] = pd.cut(test['x'], bins=x_bins, labels=False, include_lowest=True)
        test['x2_bin'] = pd.cut(test['endX'], bins=x_bins, labels=False, include_lowest=True)
        test['y1_bin'] = pd.cut(test['y'], bins=y_bins, labels=False, include_lowest=True)
        test['y2_bin'] = pd.cut(test['endY'], bins=y_bins, labels=False, include_lowest=True)
        test['xT_start'] = test[['x1_bin', 'y1_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
        test['xT_end'] = test[['x2_bin', 'y2_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
        test['xT_diff'] = test['xT_end'] - test['xT_start']
        test = test[test['xT_diff']>0].reset_index(drop=True)
        xT_added = test['xT_diff'].sum()

        xT_received_list.append([name, xT_added])
        
    else:
        xT_received_list.append([name, 0])

xT_received_home = pd.DataFrame(xT_received_list)
xT_received_home.rename({0:'playerName', 1:'xT'}, axis='columns', inplace=True)
xT_received_home = xT_received_home.sort_values(by='xT', ascending=False).reset_index(drop=True)[:3]
xT_received_home

,playerName,xT
0,Harry Kane,0.117146
1,Ben Davies,0.109752
2,Richarlison,0.077000


## Repeat for Away Team 

In [18]:
# xT via Pass
away_players = events_df[events_df['teamId'] == awayID]['playerName'].unique().tolist()
away_players = [x for x in away_players if str(x) != 'nan']

df_passes_away = events_df[(events_df['type'] == 'Pass') & (events_df['outcomeType'] == 'Successful') & 
                           (events_df['teamId'] == awayID) & 
                           (events_df['playerName'].isin(away_players))].reset_index(drop=True)


xT_list = []
for name in away_players:
    test = df_passes_away[(df_passes_away['playerName'] == name) &
                          (df_passes_away['passFreekick'] == False) & 
                          (df_passes_away['passCorner'] == False) &
                          (df_passes_away['throwIn'] == False)]
    
    if len(test) > 0:
        test['x1_bin'] = pd.cut(test['x'], bins=x_bins, labels=False, include_lowest=True)
        test['x2_bin'] = pd.cut(test['endX'], bins=x_bins, labels=False, include_lowest=True)
        test['y1_bin'] = pd.cut(test['y'], bins=y_bins, labels=False, include_lowest=True)
        test['y2_bin'] = pd.cut(test['endY'], bins=y_bins, labels=False, include_lowest=True)
        test['xT_start'] = test[['x1_bin', 'y1_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
        test['xT_end'] = test[['x2_bin', 'y2_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
        test['xT_diff'] = test['xT_end'] - test['xT_start']
        test = test[test['xT_diff']>0].reset_index(drop=True)
        xT_added = test['xT_diff'].sum()

        xT_list.append([name, xT_added])
        
    else:
        xT_list.append([name, 0])

xT_pass_away = pd.DataFrame(xT_list)
xT_pass_away.rename({0:'playerName', 1:'xT'}, axis='columns', inplace=True)
xT_pass_away = xT_pass_away.sort_values(by='xT', ascending=False).reset_index(drop=True)[:3]
xT_pass_away


# xT via Carry
xT_carry_list = []
for name in away_players:
    test = df_carries[df_carries['playerName'] == name]
    
    if len(test) > 0:
        test['x1_bin'] = pd.cut(test['x'], bins=x_bins, labels=False, include_lowest=True)
        test['x2_bin'] = pd.cut(test['endX'], bins=x_bins, labels=False, include_lowest=True)
        test['y1_bin'] = pd.cut(test['y'], bins=y_bins, labels=False, include_lowest=True)
        test['y2_bin'] = pd.cut(test['endY'], bins=y_bins, labels=False, include_lowest=True)
        test['xT_start'] = test[['x1_bin', 'y1_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
        test['xT_end'] = test[['x2_bin', 'y2_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
        test['xT_diff'] = test['xT_end'] - test['xT_start']
        test = test[test['xT_diff']>0].reset_index(drop=True)
        xT_added = test['xT_diff'].sum()

        xT_carry_list.append([name, xT_added])
        
    else:
        xT_carry_list.append([name, 0])
    
xT_carry_away = pd.DataFrame(xT_carry_list)
xT_carry_away.rename({0:'playerName', 1:'xT'}, axis='columns', inplace=True)
xT_carry_away = xT_carry_away.sort_values(by='xT', ascending=False).reset_index(drop=True)[:3]
xT_carry_away


# xT Received
df_passes_away = events_df[(events_df['type'] == 'Pass') & (events_df['outcomeType'] == 'Successful') & 
                           (events_df['teamId'] == awayID) & 
                           (events_df['playerName'].isin(away_players))].reset_index(drop=True)

xT_received_list = []
for name in away_players:
    test = df_passes_away[(df_passes_away['passRecipient'] == name) &
                          (df_passes_away['passFreekick'] == False) & 
                          (df_passes_away['passCorner'] == False) &
                          (df_passes_away['throwIn'] == False)]
    
    if len(test) > 0:
        test['x1_bin'] = pd.cut(test['x'], bins=x_bins, labels=False, include_lowest=True)
        test['x2_bin'] = pd.cut(test['endX'], bins=x_bins, labels=False, include_lowest=True)
        test['y1_bin'] = pd.cut(test['y'], bins=y_bins, labels=False, include_lowest=True)
        test['y2_bin'] = pd.cut(test['endY'], bins=y_bins, labels=False, include_lowest=True)
        test['xT_start'] = test[['x1_bin', 'y1_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
        test['xT_end'] = test[['x2_bin', 'y2_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
        test['xT_diff'] = test['xT_end'] - test['xT_start']
        test = test[test['xT_diff']>0].reset_index(drop=True)
        xT_added = test['xT_diff'].sum()

        xT_received_list.append([name, xT_added])
        
    else:
        xT_received_list.append([name, 0])

xT_received_away = pd.DataFrame(xT_received_list)
xT_received_away.rename({0:'playerName', 1:'xT'}, axis='columns', inplace=True)
xT_received_away = xT_received_away.sort_values(by='xT', ascending=False).reset_index(drop=True)[:3]
xT_received_away

,playerName,xT
0,Raheem Sterling,0.209176
1,Kai Havertz,0.196995
2,João Félix,0.109480


## Plots 

In [21]:
import matplotlib.pyplot as plt
from mplsoccer import Pitch, VerticalPitch
from matplotlib.colors import to_rgba
import matplotlib as mpl
import matplotlib.patheffects as path_effects
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap
from scipy.ndimage import gaussian_filter
import matplotlib.colors
from PIL import Image
import urllib

mpl.rcParams['figure.dpi'] = 800

FIGWIDTH, FIGHEIGHT = 14, 10
FIGSIZE = (FIGWIDTH, FIGHEIGHT)
FIG_ASPECT = FIGWIDTH / FIGHEIGHT
fig = plt.figure(figsize=FIGSIZE)

pitch1 = VerticalPitch(pitch_type='opta', pitch_color='#2B2B2B', line_color='#c7d5cc', pad_left = 40,
                       line_zorder=2, pad_bottom=20, pad_right=5,   linewidth=0.6, pitch_length=5, pad_top=30)
pitch2 = VerticalPitch(pitch_type='opta', pitch_color='#2B2B2B', line_color='#c7d5cc', pad_right = 40,
                       line_zorder=2,  pad_bottom=20, pad_left=5,  linewidth=0.6, pitch_length=5, pad_top=30)

fig.set_facecolor("#2B2B2B")

TITLE_HEIGHT = 0.1 # title axes are 10% of the figure height

#  width of pitch axes as percent of the figure width
TOP_WIDTH = 0.48
BOTTOM_WIDTH = 0.25

# calculate the horizontal space between axes (and figure sides) in percent of the figure width
TOP_SPACE = (1 - (TOP_WIDTH * 2)) / 2
BOTTOM_SPACE = (1 - (BOTTOM_WIDTH * 3)) / 2

# calculate the height of the pitch axes in percent of the figure height
height1 = (TOP_WIDTH / pitch1.ax_aspect * FIG_ASPECT)
height2 = (TOP_WIDTH / pitch2.ax_aspect * FIG_ASPECT)


# calculate pitch offsets from center / title locations
vertical_axes_space = (1 - (height1 + TITLE_HEIGHT + TITLE_HEIGHT)) / 5
bottom_offset = ((1 - height2) / 2) - vertical_axes_space
title1_bottom = 1 - vertical_axes_space - TITLE_HEIGHT
title2_bottom = 1 - (vertical_axes_space * 3) - (TITLE_HEIGHT * 2) - height1
top_offset = (1 - title1_bottom - title2_bottom - TITLE_HEIGHT) / 2



# top left
LEFT1 = TOP_SPACE
bottom1 = (1 - height1) / 2 - top_offset
ax1 = fig.add_axes((LEFT1, bottom1, TOP_WIDTH, height1))
pitch1.draw(ax=ax1)

# top right
left2 = (TOP_SPACE ) + (TOP_WIDTH)
bottom2 = (1 - height2) / 2 - top_offset
ax2 = fig.add_axes((left2, bottom2, TOP_WIDTH, height2))
pitch2.draw(ax=ax2)

ax_title2 = fig.add_axes((0, title1_bottom-0.05, 1, TITLE_HEIGHT-0.05))
ax_title2.axis('off')
ax_title2.text(left2 ,0.45,
               f'{home_name} {score} { away_name}'.upper(),
               color='white', size=13, va='center', ha='center', fontname = 'Sans Serif')
ax_title2.text(left2 ,1.1,
               'xThreat by Zone'.upper(),
               color='white', size=18, va='center', ha='center', fontname = 'Sans Serif')
ax_title2.text(left2 ,-0.15,
               f'{date_str}  |  CREATED BY ...',
               color='white', size=13, va='center', ha='center', fontname = 'Sans Serif')

fotmob_url = 'https://images.fotmob.com/image_resources/logo/teamlogo/'
team_logos2 = 8586
club_icon2 = Image.open(urllib.request.urlopen(f'{fotmob_url}{team_logos2:.0f}.png'))
newax2 = fig.add_axes([0.025, 0.85, 0.1, 0.1], anchor='NE', zorder=2)
newax2.imshow(club_icon2)
newax2.axis('off')

fotmob_url = 'https://images.fotmob.com/image_resources/logo/teamlogo/'
team_logos = 8455
club_icon = Image.open(urllib.request.urlopen(f'{fotmob_url}{team_logos:.0f}.png'))
newax = fig.add_axes([0.85, 0.85, 0.1, 0.1], anchor='NE', zorder=2)
newax.imshow(club_icon)
newax.axis('off')


    
right = 4
for i in range(9):
    pitch1.scatter(-3, 99 - right*i, s=100,
                           color=home_color, edgecolors='None',
                           marker='s', alpha=(i/10)+0.1, ax=ax1)

pitch1.annotate('Low xThreat  - - - - - -  High xThreat'.upper(), xytext=(-6,100.5), xy=(20,40),
                color='#c7d5cc',
                va='center', ha='left', fontsize=6, fontname = 'Sans Serif', ax=ax1) 


left = 4
for i in range(9):
    pitch2.scatter(-3, 33 - right*i, s=100,
                           color=away_color, edgecolors='None',
                           marker='s', alpha=(i/10)+0.1, ax=ax2)

pitch2.annotate('Low xThreat  - - - - - -  High xThreat'.upper(), xytext=(-6,34.5), xy=(20,40),
                color='#c7d5cc',
                va='center', ha='left', fontsize=6, fontname = 'Sans Serif', ax=ax2)



counter = 0
for index_x, x in enumerate(x_bins):
    for index_y, y in enumerate(y_bins):
        try:
            label_y = (y_bins[index_y] + y_bins[index_y + 1])/2
            label_x = (x_bins[index_x] + x_bins[index_x + 1])/2
            lower_y = y_bins[index_y]
            lower_x = x_bins[index_x]
            upper_y = y_bins[index_y + 1]
            upper_x = x_bins[index_x + 1]
        except:
            continue
        
        try:
            data_alpha = df_xt[df_xt['zone'] == counter]['xT_alpha'].iloc[0]
            ax1.fill_between(
                x=[lower_y, upper_y],
                y1=lower_x,
                y2=upper_x,
                color=home_color,
                zorder=0,
                alpha=data_alpha,
                ec='None'
            )
        except: 
            IndexError
        counter += 1
        

counter = 0
for index_x, x in enumerate(x_bins):
    for index_y, y in enumerate(y_bins):
        try:
            label_y = (y_bins[index_y] + y_bins[index_y + 1])/2
            label_x = (x_bins[index_x] + x_bins[index_x + 1])/2
            lower_y = y_bins[index_y]
            lower_x = x_bins[index_x]
            upper_y = y_bins[index_y + 1]
            upper_x = x_bins[index_x + 1]
        except:
            continue
        
        try:
            data_alpha = df_xt2[df_xt2['zone'] == counter]['xT_alpha'].iloc[0]
            ax2.fill_between(
                x=[lower_y, upper_y],
                y1=lower_x,
                y2=upper_x,
                color=away_color,
                zorder=0,
                alpha=data_alpha,
                ec='None'
            )
        except: 
            IndexError
        counter += 1
        

pitch1.annotate(xytext=(-3.5, 52),xy = (20,40),
                text = '> xT generated by passes & carries\n> Zones coloured according to actions start zone',
                color='#c7d5cc',
                va='center', ha='left', fontsize=7, fontname = 'Sans Serif', ax=ax1)

pitch1.annotate(text=home_name.upper(), 
                          xytext=(102, 100), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 15, ax = ax1)

pitch2.annotate(text=away_name.upper(), 
                          xytext=(102, 0), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 15, ax = ax2)


# Metrics
pitch1.annotate(text='    xThreat via Pass   '.upper(), xytext=(98, 120), xy=(60, 40), ha='center', 
                     va='center', color='white',
                     bbox=dict(facecolor=home_color, edgecolor='none', boxstyle='round,pad=.5'),
                     fontname = 'Sans Serif', fontsize=8, ax = ax1)

y = 95
i = 0
for p1, cnt in zip(xT_pass_home['playerName'], xT_pass_home['xT']):
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    cnt = round(cnt,2)
    
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    pitch1.annotate(text=p1.upper(), 
                          xytext=(y-2*i, 134), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 6, ax = ax1)
    pitch1.annotate(text=cnt, 
                          xytext=(y-2*i, 106), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 6, ax = ax1)
    
    i += 1
    
pitch1.annotate(text='   xThreat via Carry  '.upper(), xytext=(83, 120), xy=(60, 40), ha='center', va='center', color='white',
                     bbox=dict(facecolor=home_color, edgecolor='none', boxstyle='round,pad=.5'),
                     fontname = 'Sans Serif', fontsize=8, ax = ax1)
y = 80
i = 0
for p1, cnt in zip(xT_carry_home['playerName'], xT_carry_home['xT']):
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    cnt = round(cnt,2)
    
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    pitch1.annotate(text=p1.upper(), 
                          xytext=(y-2*i, 134), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 6, ax = ax1)
    pitch1.annotate(text=str(cnt), 
                          xytext=(y-2*i, 106), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 6, ax = ax1)
    
    i += 1



pitch1.annotate(text='    xThreat Received  '.upper(), xytext=(68, 120), xy=(60, 40), ha='center', va='center', color='white',
                     bbox=dict(facecolor=home_color, edgecolor='none', boxstyle='round,pad=.5'),
                     fontname = 'Sans Serif', fontsize=8, ax = ax1)
y = 65
i = 0
for p1, cnt in zip(xT_received_home['playerName'], xT_received_home['xT']):
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    cnt = round(cnt,2)
        
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    pitch1.annotate(text=p1.upper(), 
                          xytext=(y-2*i, 134), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 6, ax = ax1)
    pitch1.annotate(text=cnt, 
                          xytext=(y-2*i, 106), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 6, ax = ax1)
    
    i += 1

    
    
pitch2.annotate(text='    xThreat via Pass   '.upper(), xytext=(98, -20), xy=(60, 40), ha='center', 
                     va='center', color='white',
                     bbox=dict(facecolor=away_color, edgecolor='none', boxstyle='round,pad=.5'),
                     fontname = 'Sans Serif', fontsize=8, ax = ax2)

y = 95
i = 0
for p1, cnt in zip(xT_pass_away['playerName'], xT_pass_away['xT']):
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    cnt = round(cnt,2)

    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    pitch2.annotate(text=p1.upper(), 
                          xytext=(y-2*i, -6), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 6, ax = ax2)
    pitch2.annotate(text=cnt, 
                          xytext=(y-2*i, -34), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 6, ax = ax2)
    
    i += 1
    
pitch2.annotate(text='   xThreat via Carry  '.upper(), xytext=(83, -20), xy=(60, 40), ha='center', va='center', color='white',
                     bbox=dict(facecolor=away_color, edgecolor='none', boxstyle='round,pad=.5'),
                     fontname = 'Sans Serif', fontsize=8, ax = ax2)
y = 80
i = 0
for p1, cnt in zip(xT_carry_away['playerName'], xT_carry_away['xT']):
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    cnt = round(cnt,2)

    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    pitch2.annotate(text=p1.upper(), 
                          xytext=(y-2*i, -6), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 6, ax = ax2)
    pitch2.annotate(text=str(cnt), 
                          xytext=(y-2*i, -34), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 6, ax = ax2)
    
    i += 1



pitch2.annotate(text='    xThreat Received  '.upper(), xytext=(68.5, -20), xy=(60, 40), ha='center', va='center', color='white',
                     bbox=dict(facecolor=away_color, edgecolor='none', boxstyle='round,pad=.5'),
                     fontname = 'Sans Serif', fontsize=8, ax = ax2)
y = 65
i = 0
for p1, cnt in zip(xT_received_away['playerName'], xT_received_away['xT']):
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    cnt = round(cnt,2)
        
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    pitch2.annotate(text=p1.upper(), 
                          xytext=(y-2*i, -6), xy=(20, 40), ha='left', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 6, ax = ax2)
    pitch2.annotate(text=cnt, 
                          xytext=(y-2*i, -34), xy=(20, 40), ha='right', va='center', 
                          color='white', fontname = 'Sans Serif', fontsize = 6, ax = ax2)
    
    i += 1
    
fig.savefig('xT_zones.png', 
            bbox_inches="tight",
            edgecolor="none",
            dpi=400)